# Changing Setpoints

This notebook shows an example of changing the generator setpoints in a time-domain simulation. Data in this example is trivial, but the example can be retrofitted for scenarios such as economic dispatch incorporation or reinforcement learning.

Steps are the folllwing:

1. Initialize a system by running the power flow,
2. Set the first simulation stop time in `TDS.config.tf`,
3. Run the simulation,
3. Update the setpoints,
4. Set the new simulation stop time and repeat from 3 until the end.

## Step 1: Case Setup

In [1]:
import andes
from andes.utils import get_case

In [2]:
kundur = get_case('kundur/kundur_full.xlsx')

ss = andes.run(kundur)

Working directory: "/Users/hcui7/repos/andes/examples"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Loaded generated Python code in "/Users/hcui7/.andes/pycode".


Generated code for <PQ> is stale.


Numerical code generation (rapid incremental mode) started...


...

3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103979e-07


Converged in 5 iterations in 0.0023 seconds.


Report saved to "kundur_full_out.txt" in 0.0007 seconds.


-> Single process finished in 0.6073 seconds.


In [3]:
# disable the Toggle in this case
ss.Toggle.alter('u', 1, 0)

## Step 2: Set the First Stop Time

In [4]:
# simulate to t=1 sec

# specify the first stop in `ss.TDS.config.tf`
ss.TDS.config.tf = 1

## Step 3: Run Simulation

In [5]:
ss.TDS.run()

Initialization for dynamics completed in 0.0127 seconds.


Initialization was successful.


  0%|          | 0/100 [00:00<?, ?%/s]

Simulation to t=1.00 sec completed in 0.0116 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0007 seconds.


True

## Step 4. Apply the auxiliary power setpoints to `TGOV1.paux0.v`

First, let's check the equations of TGOV1. `ss.TGOV1.paux0` is associated with equation `0 = paux - paux0`, in which `paux` is added to the power input equation.

In [6]:
print(ss.TGOV1.doc())

Model <TGOV1> in Group <TurbineGov>
TGOV1 turbine governor model.

Implements the PSS/E TGOV1 model without deadband.

Parameters

 Name  |            Description            | Default | Unit |    Properties   
-------+-----------------------------------+---------+------+-----------------
 idx   | unique device idx                 |         |      |                 
 u     | connection status                 | 1       | bool |                 
 name  | device name                       |         |      |                 
 syn   | Synchronous generator idx         |         |      | mandatory,unique
 Tn    | Turbine power rating. Equal to    | 0       | MVA  |                 
       | `Sn` if not provided.             |         |      |                 
 wref0 | Base speed reference              | 1       | p.u. |                 
 R     | Speed regulation gain (mach. base | 0.050   | p.u. | ipower          
       | default)                          |         |      |                 


In [7]:
ss.TGOV1.paux0.v

array([0., 0., 0., 0.])

In [8]:
# look up the original values of TGOV1 make sure they are as expected

ss.TGOV1.paux0.v

array([0., 0., 0., 0.])

In [9]:
# MUST use in-place assignments. 
# Here, we increase the setpoint of the 0-th generator

# method 1: use in-place assignment again

ss.TGOV1.paux0.v[0] = 0.05

# method 2: use ``ss.TGOV1.alter()``

# ss.TGOV1.alter('paux0', 1, 0.05)

In [10]:
ss.TGOV1.paux0.v

array([0.05, 0.  , 0.  , 0.  ])

Continue to simulate to 2 seconds.

In [11]:
ss.TDS.config.tf = 2

In [12]:
ss.TDS.run()

  0%|          | 0/100 [00:00<?, ?%/s]

Simulation to t=2.00 sec completed in 0.0314 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0015 seconds.


True

In [13]:
ss.TDS.plotter.plot(ss.TGOV1.paux)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>, <Axes: xlabel='Time [s]'>)

In [14]:
ss.TDS.plotter.plot(ss.TGOV1.pout)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>, <Axes: xlabel='Time [s]'>)

In [15]:
ss.TDS.plotter.plot(ss.GENROU.omega)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>, <Axes: xlabel='Time [s]'>)

## Step 5: Set Another New Setpoints and New Ending TIme.

In this example, we clear the auxiliary power previously set to `TGOV1.paux0.v`

In [16]:
# method 1: use in-place assignment again

ss.TGOV1.paux0.v[0] = 0.

# method 2: use ``ss.TGOV1.alter()``

# ss.TGOV1.alter('paux0', 1, 0)

# set the new ending time to 10 sec.
ss.TDS.config.tf = 10

In [17]:
ss.TDS.run()

  0%|          | 0/100 [00:00<?, ?%/s]

Simulation to t=10.00 sec completed in 0.2601 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0084 seconds.


True

In [18]:
ss.TDS.plotter.plot(ss.TGOV1.paux)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>, <Axes: xlabel='Time [s]'>)

In [19]:
ss.TDS.plotter.plot(ss.GENROU.omega)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>, <Axes: xlabel='Time [s]'>)

In [20]:
!andes misc -C

"/Users/hcui7/repos/andes/examples/kundur_full_out.txt" removed.
"/Users/hcui7/repos/andes/examples/kundur_full_out.npz" removed.
"/Users/hcui7/repos/andes/examples/kundur_full_out.lst" removed.
